In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

BASE = "/kaggle/input/3w-dataset/2.0.0"
RANDOM_STATE = 42

def build_file_index(base: str) -> pd.DataFrame:
    paths = []
    for root, _, files in os.walk(base):
        for f in files:
            if f.endswith(".parquet"):
                paths.append(os.path.join(root, f))

    df = pd.DataFrame({"path": paths})

    df["event_type_code"] = df["path"].str.extract(r"/2\.0\.0/(\d+)/").astype(int)
    df["file"] = df["path"].str.split("/").str[-1]
    df["source"] = df["file"].str.extract(r"^(WELL|SIMULATED|DRAWN)")
    df["well_id"] = df["file"].str.extract(r"(WELL-\d+)")
    df["run_ts"] = df["file"].str.extract(r"_(\d{14})")
    df["run_ts"] = pd.to_datetime(df["run_ts"], format="%Y%m%d%H%M%S", errors="coerce")

    df = df.sort_values(["event_type_code","source","well_id","run_ts"]).reset_index(drop=True)
    return df

df_files = build_file_index(BASE)
df_files.head()


,path,event_type_code,file,source,well_id,run_ts
0,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201010207.parquet,WELL,WELL-00001,2017-02-01 01:02:07
1,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201060114.parquet,WELL,WELL-00001,2017-02-01 06:01:14
2,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201110124.parquet,WELL,WELL-00001,2017-02-01 11:01:24
3,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201160311.parquet,WELL,WELL-00001,2017-02-01 16:03:11
4,/kaggle/input/3w-dataset/2.0.0/0/WELL-00001_20...,0,WELL-00001_20170201210228.parquet,WELL,WELL-00001,2017-02-01 21:02:28


In [3]:
print("Num files:", len(df_files))
display(df_files["source"].value_counts(dropna=False))
display(df_files["event_type_code"].value_counts().sort_index())


Num files: 2228


source
WELL         1119
SIMULATED    1089
DRAWN          20
Name: count, dtype: int64

event_type_code
0    594
1    128
2     38
3    106
4    343
5    450
6    221
7     46
8     95
9    207
Name: count, dtype: int64

In [4]:
VAR_RENAME = {
    "ABER-CKGL": "gl_choke_opening_pct",
    "ABER-CKP":  "prod_choke_opening_pct",
    "ESTADO-DHSV":   "dhsv_state",
    "ESTADO-M1":     "prod_master_valve_state",
    "ESTADO-M2":     "ann_master_valve_state",
    "ESTADO-PXO":    "pig_crossover_valve_state",
    "ESTADO-SDV-GL": "gl_shutdown_valve_state",
    "ESTADO-SDV-P":  "prod_shutdown_valve_state",
    "ESTADO-W1":     "prod_wing_valve_state",
    "ESTADO-W2":     "ann_wing_valve_state",
    "ESTADO-XO":     "crossover_valve_state",
    "P-ANULAR":     "annulus_pressure_pa",
    "P-JUS-BS":     "svc_pump_downstream_pressure_pa",
    "P-JUS-CKGL":   "gl_choke_downstream_pressure_pa",
    "P-JUS-CKP":    "prod_choke_downstream_pressure_pa",
    "P-MON-CKGL":   "gl_choke_upstream_pressure_pa",
    "P-MON-CKP":    "prod_choke_upstream_pressure_pa",
    "P-MON-SDV-P":  "prod_sdv_upstream_pressure_pa",
    "P-PDG":        "pdg_downhole_pressure_pa",
    "PT-P":         "xmas_tree_prod_line_pressure_pa",
    "P-TPT":        "tpt_pressure_pa",
    "QBS": "svc_pump_flow_m3s",
    "QGL": "gas_lift_flow_m3s",
    "T-JUS-CKP": "prod_choke_downstream_temp_c",
    "T-MON-CKP": "prod_choke_upstream_temp_c",
    "T-PDG":     "pdg_downhole_temp_c",
    "T-TPT":     "tpt_temp_c",
    "class": "class_code",
    "state": "state_code",
}

EVENT_TYPE_CODE_TO_NAME = {
    0:"Normal Operation", 1:"Abrupt Increase of BSW", 2:"Spurious Closure of DHSV",
    3:"Severe Slugging", 4:"Flow Instability", 5:"Rapid Productivity Loss",
    6:"Quick Restriction in PCK", 7:"Scaling in PCK",
    8:"Hydrate in Production Line", 9:"Hydrate in Service Line",
}

LABEL_COLS = {"class_code", "state_code", "class_label", "state_label"}

def clean_3w_instance(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Ensure timestamp index سواء كان index أو column
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
        df = df.set_index("timestamp")
    else:
        df.index = pd.to_datetime(df.index, errors="coerce")

    df = df[~df.index.isna()].sort_index()
    df.index.name = "timestamp"

    # rename
    df = df.rename(columns=VAR_RENAME)

    # numeric coercion
    for c in df.columns:
        if c in ("class_code", "state_code"):
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int16")
        else:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")

    return df


In [5]:
def summarize_timeseries_fast(df_clean: pd.DataFrame) -> dict:
    sensors = df_clean.drop(columns=list(LABEL_COLS), errors="ignore")
    num = sensors.select_dtypes(include=[np.number])

    out = {
        "n_obs": int(len(df_clean)),
        "duration_s": float((df_clean.index.max() - df_clean.index.min()).total_seconds())
                      if len(df_clean) else np.nan,
    }

    if num.shape[1] == 0 or len(num) == 0:
        return out

    agg = num.agg(["mean", "std", "min", "max"]).T
    last = num.iloc[-1]
    miss = num.isna().mean()
    frozen = (num.nunique(dropna=True) <= 1).astype("int8")

    for col in num.columns:
        out[f"{col}__mean"] = agg.loc[col, "mean"]
        out[f"{col}__std"]  = agg.loc[col, "std"]
        out[f"{col}__min"]  = agg.loc[col, "min"]
        out[f"{col}__max"]  = agg.loc[col, "max"]
        out[f"{col}__last"] = last[col]
        out[f"{col}__missing_frac"] = miss[col]
        out[f"{col}__is_frozen"] = int(frozen[col])

    return out


In [6]:
def build_row_per_file_dataset(df_files: pd.DataFrame, n_files: int = 500, random_state: int = 42) -> pd.DataFrame:
    sample = df_files.sample(n_files, random_state=random_state).reset_index(drop=True)

    rows = []
    for _, r in tqdm(sample.iterrows(), total=len(sample), desc="Building row-per-file dataset"):
        df_raw = pd.read_parquet(r["path"])
        df_clean = clean_3w_instance(df_raw)

        feats = summarize_timeseries_fast(df_clean)

        feats["event_type_code"] = int(r["event_type_code"])
        feats["event_type_name"] = EVENT_TYPE_CODE_TO_NAME.get(int(r["event_type_code"]), "Unknown")
        feats["source"] = r["source"]
        feats["well_id"] = r["well_id"]
        feats["run_ts"] = r["run_ts"]
        feats["file"] = r["file"]
        rows.append(feats)

    return pd.DataFrame(rows)

df_ml = build_row_per_file_dataset(df_files, n_files=200, random_state=RANDOM_STATE)
df_ml.head()


Building row-per-file dataset:   0%|          | 0/200 [00:00<?, ?it/s]

,n_obs,duration_s,gl_choke_opening_pct__mean,gl_choke_opening_pct__std,gl_choke_opening_pct__min,gl_choke_opening_pct__max,gl_choke_opening_pct__last,gl_choke_opening_pct__missing_frac,gl_choke_opening_pct__is_frozen,prod_choke_opening_pct__mean,...,tpt_temp_c__max,tpt_temp_c__last,tpt_temp_c__missing_frac,tpt_temp_c__is_frozen,event_type_code,event_type_name,source,well_id,run_ts,file
0,21421,21420.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,...,116.793701,116.685699,0.000000,0,0,Normal Operation,WELL,WELL-00001,2017-05-27 15:00:00,WELL-00001_20170527150000.parquet
1,21355,21354.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,...,116.520302,116.417702,0.000000,0,0,Normal Operation,WELL,WELL-00006,2017-08-20 00:00:00,WELL-00006_20170820000000.parquet
2,26999,26998.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,...,NaN,NaN,1.000000,1,6,Quick Restriction in PCK,SIMULATED,NaN,NaT,SIMULATED_00138.parquet
3,21470,21469.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,...,118.707703,118.698502,0.000000,0,0,Normal Operation,WELL,WELL-00002,2017-06-18 16:02:11,WELL-00002_20170618160211.parquet
4,30601,30600.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,...,111.813499,3.275891,0.064736,0,2,Spurious Closure of DHSV,WELL,WELL-00011,2014-05-15 08:30:00,WELL-00011_20140515083000.parquet


In [7]:
print(df_ml.shape)
display(df_ml["event_type_name"].value_counts())


(200, 197)


event_type_name
Normal Operation              64
Rapid Productivity Loss       42
Flow Instability              25
Hydrate in Service Line       21
Quick Restriction in PCK      15
Abrupt Increase of BSW        11
Severe Slugging               10
Hydrate in Production Line     7
Scaling in PCK                 4
Spurious Closure of DHSV       1
Name: count, dtype: int64